In [42]:
import cv2 
from collections import Counter
import numpy as np 
from tqdm import tqdm 

In [57]:
import os

def read_jpg_files(directory):
    jpg_files = []
    
    # Walk through the directory and subdirectories
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.jpg') and 'rot' not in file:
                # Get the full path to the .jpg file
                file_path = os.path.join(root, file)
                jpg_files.append(file_path)

    return jpg_files

# Example: Provide the directory you want to search in
directory_path = "/kaggle/input/facial-emotion-expressions/images/validation"
trainpaths = read_jpg_files(directory_path)
real=np.load('/kaggle/input/realnp/real.npy')

In [58]:
labels=[trainpaths[i].split('/')[-2] for i in range(len(trainpaths))]

In [59]:
from transformers import pipeline
from tqdm import tqdm 
from PIL import Image

# Initialize the pipeline
pipe = pipeline("image-classification", model="dima806/facial_emotions_image_detection",device=0)

# List of image paths
image_paths = trainpaths

# Process images and classify them
results = []
for path in tqdm(image_paths):
    # Load the image
    image = Image.open(path)
    
    # Use the pipeline to classify the image
    result = pipe(image)
    
    # Get the emotion with the maximum score
    max_emotion = max(result, key=lambda x: x['score'])
    
    # Store the result
    results.append(max_emotion)

100%|██████████| 7066/7066 [01:51<00:00, 63.31it/s]


In [61]:
preds=[results[i]['label'] for i in range(len(results))]
Counter(preds),Counter(labels)

(Counter({'happy': 1805,
          'neutral': 1193,
          'sad': 1175,
          'fear': 997,
          'angry': 951,
          'surprise': 823,
          'disgust': 122}),
 Counter({'happy': 1825,
          'neutral': 1216,
          'sad': 1139,
          'fear': 1018,
          'angry': 960,
          'surprise': 797,
          'disgust': 111}))

In [64]:
final_preds=[]
for i in range(len(preds)):
    if preds[i]=='angry':
        final_preds.append('anger')
    else:
        final_preds.append(preds[i])

In [65]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
print(classification_report(labels,preds))

              precision    recall  f1-score   support

       angry       0.87      0.86      0.87       960
     disgust       0.91      1.00      0.95       111
        fear       0.85      0.83      0.84      1018
       happy       0.95      0.94      0.94      1825
     neutral       0.86      0.85      0.86      1216
         sad       0.80      0.83      0.81      1139
    surprise       0.90      0.93      0.91       797

    accuracy                           0.88      7066
   macro avg       0.88      0.89      0.88      7066
weighted avg       0.88      0.88      0.88      7066

